In [1]:
%load_ext autoreload
%autoreload 3

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
import networkx as nx
import scipy
import sklearn
import time
import pickle
import warnings

# methods
from gaccord import GraphicalAccord, GraphicalConcord
from inverse_covariance import QuicGraphicalLasso
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri

# utils
import sys
sys.path.append('../utils')
from utils import standardize, partial_corr, tp_fp, mcc, precision_recall, pseudo_BIC, gauss_BIC, proj_precision_mat, newton_one_step
from generate_graphs import generate_erdos_renyi, generate_data

In [2]:
def compute_norm(Theta, Theta_hat):
    p = len(Theta)
    Theta_bool = np.where(Theta != 0, 1, 0)
    Theta_hat_bool = np.where(Theta_hat != 0, 1, 0)

    mask = np.tri(p, p, k=-1, dtype=bool)
    edge_true = Theta_bool[mask]
    edge_hat = Theta_hat_bool[mask]
    TP_indx = np.where((edge_true == 1) & (edge_hat == 1) == 1)[0]
    FP_indx = np.where((edge_true == 0) & (edge_hat == 1) == 1)[0]
    FN_indx = np.where((edge_true == 1) & (edge_hat == 0) == 1)[0]

    theta_true = Theta[mask]
    theta_hat = Theta_hat[mask]

    # norm_TP = np.linalg.norm(theta_true[TP_indx] - theta_hat[TP_indx])
    # norm_FP = np.linalg.norm(theta_true[FP_indx] - theta_hat[FP_indx])
    # norm_FN = np.linalg.norm(theta_true[FN_indx] - theta_hat[FN_indx])
    # norm_diag = np.linalg.norm(np.diag(Theta) - np.diag(Theta_hat))
    norm_total = np.linalg.norm(theta_true - theta_hat)
    norm_TP_average = np.sqrt(np.mean((theta_true[TP_indx]-theta_hat[TP_indx])**2))
    norm_FP_average = np.sqrt(np.mean((theta_true[FP_indx]-theta_hat[FP_indx])**2))
    norm_FN_average = np.sqrt(np.mean((theta_true[FN_indx]-theta_hat[FN_indx])**2))
    norm_diag_average = np.sqrt(np.mean((np.diag(Theta) - np.diag(Theta_hat))**2))

    return norm_total, norm_TP_average, norm_FP_average, norm_FN_average, norm_diag_average

In [3]:
def partial_corr_to_precision(Rho, diagonal_values):
    p = len(Rho)
    Theta = np.zeros_like(Rho)
    
    # Compute the off-diagonal elements of the precision matrix
    for i in range(p):
        for j in range(i + 1, p):
            Theta[i, j] = -Rho[i, j] / np.sqrt(diagonal_values[i] * diagonal_values[j])
            Theta[j, i] = Theta[i, j]
    
    # Fill the diagonal elements with the provided diagonal values
    np.fill_diagonal(Theta, diagonal_values)
    
    return Theta

In [4]:
# select graph structure = ['hub_network', 'erdos_renyi']
graph_structure = 'erdos_renyi'

### Generate graph and data

In [5]:
if graph_structure == 'hub_network':
    # we use pre-made hub-network graph structure, which was constructed by the following procedure:
    # (1) create a Barabasi-Albert scale-free graph
    # (2) randomly choose 5% of the nodes to be hub nodes
    # (3) for each hub node, construct a complete sub-graph (clique)
    Skel = np.genfromtxt('../data/hub_network_structure.txt', delimiter=',')

    n, p = 500, 1000
    n_prop_to_p = [0.5]
    random_state = 2023
    lower_weight, upper_weight = 0.5, 1.0

    # projection method
    np.random.seed(random_state)
    edge_weights = np.random.uniform(low=lower_weight, high=upper_weight, size=(p,p))
    edge_signs = np.random.choice([-1,1], size=(p,p))
    Theta = np.multiply(edge_weights, edge_signs)
    Theta = np.multiply(Skel, Theta)
    Theta = np.tril(Theta) + np.tril(Theta).T
    nz_indx = np.nonzero(Theta)
    for i in range(100):
        Theta = proj_precision_mat(Theta, nz_indx)
        if np.linalg.cond(Theta) < 20:
            break

    Theta = np.real(Theta)
    # spread diagonal of precision matrix
    spread_diag=[1, 3]
    d = np.random.uniform(spread_diag[0], spread_diag[1], p)
    Theta = np.diag(d) @ Theta @ np.diag(d)
    Rho = partial_corr(Theta)
    Sigma = np.linalg.inv(Theta)

    Xs = generate_data(p, n_prop_to_p, Sigma, N=1, standardize=False, random_state=2023)
    X = Xs[0]

elif graph_structure == 'erdos_renyi':
    n, p = 500, 1000
    n_prop_to_p = [0.5]
    Theta, Sigma = generate_erdos_renyi(p, type='proj', edge_prob=0.01, lower_weight=0.5, upper_weight=1.0, spread_diag=[1, 3], random_state=2023)
    Rho = partial_corr(Theta)

    Xs = generate_data(p, n_prop_to_p, Sigma, N=1, standardize=False, random_state=2023)
    X = Xs[0]

### Run ACCORD

In [6]:
accord_p_bics, accord_ep_bics_02 = [], []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_accord = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_accord = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_accord:
    # ACCORD
    model = GraphicalAccord(Omega_star=np.eye(p), lam1=lam, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Omega_hat = model.omega_.toarray()
    Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
    
    accord_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    accord_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_accord = lams_accord[np.argmin(accord_p_bics)]
print('Best lam (based on pbic):', best_lam_accord)
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

total_norm_accord_pbic, TP_average_norm_accord_pbic, FP_average_norm_accord_pbic, FN_average_norm_accord_pbic, diag_average_norm_accord_pbic = compute_norm(Theta, Theta_hat)
# total_norm_accord_pbic, TP_average_norm_accord_pbic, FP_average_norm_accord_pbic, FN_average_norm_accord_pbic, diag_average_norm_accord_pbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_accord_pbic = np.linalg.norm(Theta_hat - Theta)

# use optimal lambda based on ep-bic
best_lam_accord = lams_accord[np.argmin(accord_ep_bics_02)]
print('Best lam (based on epbic):', best_lam_accord)
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

total_norm_accord_epbic, TP_average_norm_accord_epbic, FP_average_norm_accord_epbic, FN_average_norm_accord_epbic, diag_average_norm_accord_epbic = compute_norm(Theta, Theta_hat)
# total_norm_accord_epbic, TP_average_norm_accord_epbic, FP_average_norm_accord_epbic, FN_average_norm_accord_epbic, diag_average_norm_accord_epbic = compute_norm(Rho, Rho_hat)
# theta_fro_norm_accord_epbic = np.linalg.norm(Theta_hat - Theta)

Best lam (based on pbic): 0.09113479729059223
Best lam (based on epbic): 0.12520315315081312


### Run ACCORD-SP (version 1, average nonzeros)

In [7]:
accord_sp1_p_bics, accord_sp1_ep_bics_02 = [], []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_accord_sp1 = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_accord_sp1 = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_accord_sp1:
    # ACCORD
    model = GraphicalAccord(Omega_star=np.eye(p), lam1=lam, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Omega_hat = model.omega_.toarray()

    # set w_ij = w_ji = 0 if at least one of them is 0 and average nonzero values
    zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
    Omega_hat[zero_indices] = 0
    Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
    
    accord_sp1_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    accord_sp1_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_accord = lams_accord[np.argmin(accord_sp1_p_bics)]
print('Best lam (based on pbic):', best_lam_accord)
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
# set w_ij = w_ji = 0 if at least one of them is 0 and average nonzero values
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

total_norm_accord_sp1_pbic, TP_average_norm_accord_sp1_pbic, FP_average_norm_accord_sp1_pbic, FN_average_norm_accord_sp1_pbic, diag_average_norm_accord_sp1_pbic = compute_norm(Theta, Theta_hat)
# total_norm_accord_sp1_pbic, TP_average_norm_accord_sp1_pbic, FP_average_norm_accord_sp1_pbic, FN_average_norm_accord_sp1_pbic, diag_average_norm_accord_sp1_pbic = compute_norm(Rho, Rho_hat)

# use optimal lambda based on ep-bic
best_lam_accord = lams_accord[np.argmin(accord_sp1_ep_bics_02)]
print('Best lam (based on epbic):', best_lam_accord)
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
# set w_ij = w_ji = 0 if at least one of them is 0 and average nonzero values
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
Theta_hat = 0.5 * ((np.diag(np.diag(Omega_hat)) @ Omega_hat) + (Omega_hat.T @ np.diag(np.diag(Omega_hat))))
Rho_hat = partial_corr(Theta_hat)

total_norm_accord_sp1_epbic, TP_average_norm_accord_sp1_epbic, FP_average_norm_accord_sp1_epbic, FN_average_norm_accord_sp1_epbic, diag_average_norm_accord_sp1_epbic = compute_norm(Theta, Theta_hat)
# total_norm_accord_sp1_epbic, TP_average_norm_accord_sp1_epbic, FP_average_norm_accord_sp1_epbic, FN_average_norm_accord_sp1_epbic, diag_average_norm_accord_sp1_epbic = compute_norm(Rho, Rho_hat)

Best lam (based on pbic): 0.07775327928489881
Best lam (based on epbic): 0.07775327928489881


### Run ACCORD-SP (version 2, take a smaller magnitude)

In [8]:
accord_sp2_p_bics, accord_sp2_ep_bics_02 = [], []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_accord_sp2 = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_accord_sp2 = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_accord_sp2:
    # ACCORD
    model = GraphicalAccord(Omega_star=np.eye(p), lam1=lam, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Omega_hat = model.omega_.toarray()

    # set w_ij = w_ji = 0 if at least one of them is 0 and choose the smaller nonzero otherwise
    zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
    Omega_hat[zero_indices] = 0
    nonzero_indices = np.where((Omega_hat != 0) & (Omega_hat.T != 0))
    for i, j in zip(*nonzero_indices):
        if abs(Omega_hat[i, j]) > abs(Omega_hat[j, i]):
            Omega_hat[i, j] = Omega_hat[j, i]
        else:
            Omega_hat[j, i] = Omega_hat[i, j]
    Theta_hat = Omega_hat
    
    accord_sp2_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    accord_sp2_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_accord = lams_accord[np.argmin(accord_sp2_p_bics)]
print('Best lam (based on pbic):', best_lam_accord)
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
# set w_ij = w_ji = 0 if at least one of them is 0 and choose the smaller nonzero otherwise
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
nonzero_indices = np.where((Omega_hat != 0) & (Omega_hat.T != 0))
for i, j in zip(*nonzero_indices):
    if abs(Omega_hat[i, j]) > abs(Omega_hat[j, i]):
        Omega_hat[i, j] = Omega_hat[j, i]
    else:
        Omega_hat[j, i] = Omega_hat[i, j]
Theta_hat = Omega_hat
Rho_hat = partial_corr(Theta_hat)

total_norm_accord_sp2_pbic, TP_average_norm_accord_sp2_pbic, FP_average_norm_accord_sp2_pbic, FN_average_norm_accord_sp2_pbic, diag_average_norm_accord_sp2_pbic = compute_norm(Theta, Theta_hat)
# total_norm_accord_sp2_pbic, TP_average_norm_accord_sp2_pbic, FP_average_norm_accord_sp2_pbic, FN_average_norm_accord_sp2_pbic, diag_average_norm_accord_sp2_pbic = compute_norm(Rho, Rho_hat)

# use optimal lambda based on ep-bic
best_lam_accord = lams_accord[np.argmin(accord_sp2_ep_bics_02)]
print('Best lam (based on epbic):', best_lam_accord)
model = GraphicalAccord(Omega_star=np.eye(p), lam1=best_lam_accord, stepsize_multiplier=1.0, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Omega_hat = model.omega_.toarray()
# set w_ij = w_ji = 0 if at least one of them is 0 and choose the smaller nonzero otherwise
zero_indices = np.where((Omega_hat == 0) | (Omega_hat.T == 0))
Omega_hat[zero_indices] = 0
nonzero_indices = np.where((Omega_hat != 0) & (Omega_hat.T != 0))
for i, j in zip(*nonzero_indices):
    if abs(Omega_hat[i, j]) > abs(Omega_hat[j, i]):
        Omega_hat[i, j] = Omega_hat[j, i]
    else:
        Omega_hat[j, i] = Omega_hat[i, j]
Theta_hat = Omega_hat
Rho_hat = partial_corr(Theta_hat)

total_norm_accord_sp2_epbic, TP_average_norm_accord_sp2_epbic, FP_average_norm_accord_sp2_epbic, FN_average_norm_accord_sp2_epbic, diag_average_norm_accord_sp2_epbic = compute_norm(Theta, Theta_hat)
# total_norm_accord_sp2_epbic, TP_average_norm_accord_sp2_epbic, FP_average_norm_accord_sp2_epbic, FN_average_norm_accord_sp2_epbic, diag_average_norm_accord_sp2_epbic = compute_norm(Rho, Rho_hat)

Best lam (based on pbic): 0.07775327928489881
Best lam (based on epbic): 0.07775327928489881


### Run CONCORD

In [9]:
concord_p_bics, concord_ep_bics_02 = [], []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_concord = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_concord = lams[::-1][8:20]

random_state = 2023
np.random.seed(random_state)
for lam in lams_concord:
    # CONCORD
    model = GraphicalConcord(Omega_star=np.eye(p), lam1=lam, backtracking=True, epstol=1e-7, maxitr=100)
    model.fit(X)
    Theta_hat = model.omega_.toarray()
    
    concord_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    concord_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_concord = lams_concord[np.argmin(concord_p_bics)]
print('Best lam (based on pbic):', best_lam_concord)
model = GraphicalConcord(Omega_star=np.eye(p), lam1=best_lam_concord, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Theta_hat = model.omega_.toarray()
Rho_hat = partial_corr(Theta_hat)

total_norm_concord_pbic, TP_average_norm_concord_pbic, FP_average_norm_concord_pbic, FN_average_norm_concord_pbic, diag_average_norm_concord_pbic = compute_norm(Theta, Theta_hat)
# total_norm_concord_pbic, TP_average_norm_concord_pbic, FP_average_norm_concord_pbic, FN_average_norm_concord_pbic, diag_average_norm_concord_pbic = compute_norm(Rho, Rho_hat)

# use optimal lambda based on ep-bic
best_lam_concord = lams_concord[np.argmin(concord_ep_bics_02)]
print('Best lam (based on epbic):', best_lam_concord)
model = GraphicalConcord(Omega_star=np.eye(p), lam1=best_lam_concord, backtracking=True, epstol=1e-7, maxitr=100)
model.fit(X)
Theta_hat = model.omega_.toarray()
Rho_hat = partial_corr(Theta_hat)

total_norm_concord_epbic, TP_average_norm_concord_epbic, FP_average_norm_concord_epbic, FN_average_norm_concord_epbic, diag_average_norm_concord_epbic = compute_norm(Theta, Theta_hat)
# total_norm_concord_epbic, TP_average_norm_concord_epbic, FP_average_norm_concord_epbic, FN_average_norm_concord_epbic, diag_average_norm_concord_epbic = compute_norm(Rho, Rho_hat)

Best lam (based on pbic): 0.07775327928489881
Best lam (based on epbic): 0.09866587928096081


### Run Glasso

In [10]:
%%capture

glasso_g_bics = []

S = np.matmul(X.T, X)/n
S.flat[::S.shape[0] + 1] = 0
lam_max = np.max(np.abs(S))
lam_min = 0.1 * lam_max
lams = np.logspace(np.log10(lam_min), np.log10(lam_max), 30)
if graph_structure == 'hub_network':
    lams_glasso = lams[::-1][15:27]
elif graph_structure == 'erdos_renyi':
    lams_glasso = lams[::-1][12:24] # pre-selected lambdas for erdos renyi (diag=[1, 3])

random_state = 2023
np.random.seed(random_state)
for lam in lams_glasso:
    quic = QuicGraphicalLasso(lam=lam, max_iter=100, init_method='cov', auto_scale=False).fit(X)
    Theta_hat = quic.precision_

    glasso_g_bics.append(gauss_BIC(X, Theta_hat))

# use optimal lambda based on g-bic
best_lam_glasso = lams_glasso[np.argmin(glasso_g_bics)]
print('Best lam (based on gbic):', best_lam_glasso)
quic = QuicGraphicalLasso(lam=best_lam_glasso, max_iter=100, init_method='cov', auto_scale=False).fit(X)
Theta_hat = quic.precision_
Rho_hat = partial_corr(Theta_hat)

total_norm_glasso, TP_average_norm_glasso, FP_average_norm_glasso, FN_average_norm_glasso, diag_average_norm_glasso = compute_norm(Theta, Theta_hat)
# total_norm_glasso, TP_average_norm_glasso, FP_average_norm_glasso, FN_average_norm_glasso, diag_average_norm_glasso = compute_norm(Rho, Rho_hat)

### Run SPACE

In [11]:
%%capture

# import SPACE
rpy2.robjects.numpy2ri.activate()
space = importr('space')

space_p_bics, space_ep_bics_02 = [], []

if graph_structure == 'hub_network':
    lams_space = np.logspace(np.log10(40), np.log10(200), 12)
elif graph_structure == 'erdos_renyi':
    lams_space = np.logspace(np.log10(40), np.log10(200), 12) # pre-selected lambdas for erdos renyi (diag=[1,3])

random_state = 2023
np.random.seed(random_state)
for lam in lams_space:
    prec = space.space_joint(X, np.array([lam]))
    Theta_hat = np.array(prec[0])

    space_p_bics.append(pseudo_BIC(X, Theta_hat, modified=False))
    space_ep_bics_02.append(pseudo_BIC(X, Theta_hat, modified=True, gamma=0.2))

# use optimal lambda based on p-bic
best_lam_space = lams_space[np.argmin(space_p_bics)]
print('Best lam (based on pbic):', best_lam_space)
prec = space.space_joint(X, np.array([best_lam_space]))
Theta_hat = partial_corr_to_precision(prec[0], prec[1])
Rho_hat = np.array(prec[0])

total_norm_space_pbic, TP_average_norm_space_pbic, FP_average_norm_space_pbic, FN_average_norm_space_pbic, diag_average_norm_space_pbic = compute_norm(Theta, Theta_hat)
# total_norm_space_pbic, TP_average_norm_space_pbic, FP_average_norm_space_pbic, FN_average_norm_space_pbic, diag_average_norm_space_pbic = compute_norm(Rho, Rho_hat)

# use optimal lambda based on ep-bic
best_lam_space = lams_space[np.argmin(space_ep_bics_02)]
print('Best lam (based on epbic):', best_lam_space)
prec = space.space_joint(X, np.array([best_lam_space]))
Theta_hat = partial_corr_to_precision(prec[0], prec[1])
Rho_hat = np.array(prec[0])

total_norm_space_epbic, TP_average_norm_space_epbic, FP_average_norm_space_epbic, FN_average_norm_space_epbic, diag_average_norm_space_epbic = compute_norm(Theta, Theta_hat)
# total_norm_space_epbic, TP_average_norm_space_epbic, FP_average_norm_space_epbic, FN_average_norm_space_epbic, diag_average_norm_space_epbic = compute_norm(Rho, Rho_hat)

In [14]:
# hub network
print('ACCORD')

print('Total norm (based on p-bic):', total_norm_accord_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_accord_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_accord_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_accord_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_accord_epbic)
print('Diagonal average norm (based on ep-bic):', diag_average_norm_accord_epbic, '\n')

print('ACCORD (sparsified & averaged)')

print('Total norm (based on p-bic):', total_norm_accord_sp1_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_sp1_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_sp1_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_accord_sp1_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_accord_sp1_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_accord_sp1_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_sp1_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_sp1_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_accord_sp1_epbic)
print('Diagonal average norm (based on ep-bic):', diag_average_norm_accord_sp1_epbic, '\n')

print('ACCORD (sparsified & smaller nonzeros taken)')

print('Total norm (based on p-bic):', total_norm_accord_sp2_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_sp2_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_sp2_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_accord_sp2_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_accord_sp2_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_accord_sp2_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_sp2_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_sp2_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_accord_sp2_epbic)
print('Diagonal average norm (based on ep-bic):', diag_average_norm_accord_sp2_epbic, '\n')

print('CONCORD')

print('Total norm (based on p-bic):', total_norm_concord_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_concord_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_concord_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_concord_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_concord_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_concord_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_concord_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_concord_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_concord_epbic)
print('Diagonal average norm (based on ep-bic):', diag_average_norm_concord_epbic, '\n')

print('Glasso')

print('Total norm (based on g-bic):', total_norm_glasso)
print('TP average norm (based on g-bic):', TP_average_norm_glasso)
print('FP average norm (based on g-bic):', FP_average_norm_glasso)
print('FN average norm (based on g-bic):', FN_average_norm_glasso)
print('Diagonal average norm (based on g-bic):', diag_average_norm_glasso, '\n')

print('SPACE')

print('Total norm (based on p-bic):', total_norm_space_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_space_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_space_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_space_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_space_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_space_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_space_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_space_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_space_epbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_space_epbic, '\n')

ACCORD
Total norm (based on p-bic): 64.81247477792128
TP average norm (based on p-bic): 1.1613028835974477
FP average norm (based on p-bic): 0.014140201650008679
FN average norm (based on p-bic): 0.934125187848966
Diagonal average norm (based on p-bic): 1.965793270528603 

Total norm (based on ep-bic): 66.7386030034716
TP average norm (based on ep-bic): 1.2016387812373441
FP average norm (based on ep-bic): 0.011209297009995565
FN average norm (based on ep-bic): 0.9968469396496907
Diagonal average norm (based on ep-bic): 2.129521833489333 

ACCORD (sparsified & averaged)
Total norm (based on p-bic): 57.06751802582949
TP average norm (based on p-bic): 1.0538630506528033
FP average norm (based on p-bic): 0.040454113660555104
FN average norm (based on p-bic): 0.7497041336244455
Diagonal average norm (based on p-bic): 1.465960041801372 

Total norm (based on ep-bic): 57.06751802582949
TP average norm (based on ep-bic): 1.0538630506528033
FP average norm (based on ep-bic): 0.0404541136605551

In [12]:
# erdos renyi
print('ACCORD')

print('Total norm (based on p-bic):', total_norm_accord_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_accord_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_accord_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_accord_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_accord_epbic)
print('Diagonal average norm (based on ep-bic):', diag_average_norm_accord_epbic, '\n')

print('ACCORD (sparsified & averaged)')

print('Total norm (based on p-bic):', total_norm_accord_sp1_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_sp1_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_sp1_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_accord_sp1_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_accord_sp1_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_accord_sp1_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_sp1_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_sp1_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_accord_sp1_epbic)
print('Diagonal average norm (based on ep-bic):', diag_average_norm_accord_sp1_epbic, '\n')

print('ACCORD (sparsified & smaller nonzeros taken)')

print('Total norm (based on p-bic):', total_norm_accord_sp2_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_accord_sp2_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_accord_sp2_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_accord_sp2_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_accord_sp2_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_accord_sp2_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_accord_sp2_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_accord_sp2_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_accord_sp2_epbic)
print('Diagonal average norm (based on ep-bic):', diag_average_norm_accord_sp2_epbic, '\n')

print('CONCORD')

print('Total norm (based on p-bic):', total_norm_concord_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_concord_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_concord_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_concord_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_concord_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_concord_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_concord_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_concord_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_concord_epbic)
print('Diagonal average norm (based on ep-bic):', diag_average_norm_concord_epbic, '\n')

print('Glasso')

print('Total norm (based on g-bic):', total_norm_glasso)
print('TP average norm (based on g-bic):', TP_average_norm_glasso)
print('FP average norm (based on g-bic):', FP_average_norm_glasso)
print('FN average norm (based on g-bic):', FN_average_norm_glasso)
print('Diagonal average norm (based on g-bic):', diag_average_norm_glasso, '\n')

print('SPACE')

print('Total norm (based on p-bic):', total_norm_space_pbic)
print('TP average norm (based on p-bic):', TP_average_norm_space_pbic)
print('FP average norm (based on p-bic):', FP_average_norm_space_pbic)
print('FN average norm (based on p-bic):', FN_average_norm_space_pbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_space_pbic, '\n')

print('Total norm (based on ep-bic):', total_norm_space_epbic)
print('TP average norm (based on ep-bic):', TP_average_norm_space_epbic)
print('FP average norm (based on ep-bic):', FP_average_norm_space_epbic)
print('FN average norm (based on ep-bic):', FN_average_norm_space_epbic)
print('Diagonal average norm (based on p-bic):', diag_average_norm_space_epbic, '\n')

ACCORD
Total norm (based on p-bic): 40.3284753961415
TP average norm (based on p-bic): 0.5217747984606519
FP average norm (based on p-bic): 0.018633766066640543
FN average norm (based on p-bic): 0.6891002782534517
Diagonal average norm (based on p-bic): 1.7476712160979009 

Total norm (based on ep-bic): 43.643103300051145
TP average norm (based on ep-bic): 0.48520449994104975
FP average norm (based on ep-bic): 0.012938777648971765
FN average norm (based on ep-bic): 0.7033813400629167
Diagonal average norm (based on ep-bic): 2.1216316114182656 

ACCORD (sparsified & averaged)
Total norm (based on p-bic): 38.445270131195734
TP average norm (based on p-bic): 0.47306570516003194
FP average norm (based on p-bic): 0.03781995968504634
FN average norm (based on p-bic): 0.6324606314733722
Diagonal average norm (based on p-bic): 1.5241025831437907 

Total norm (based on ep-bic): 38.445270131195734
TP average norm (based on ep-bic): 0.47306570516003194
FP average norm (based on ep-bic): 0.0378199

### Confirm that total^2 = TP^2 + FP^2 + FN^2

In [ ]:
def compute_norm_test(Theta, Theta_hat):
    p = len(Theta)
    Theta_bool = np.where(Theta != 0, 1, 0)
    Theta_hat_bool = np.where(Theta_hat != 0, 1, 0)

    mask = np.tri(p, p, k=-1, dtype=bool)
    edge_true = Theta_bool[mask]
    edge_hat = Theta_hat_bool[mask]
    TP_indx = np.where((edge_true == 1) & (edge_hat == 1) == 1)[0]
    FP_indx = np.where((edge_true == 0) & (edge_hat == 1) == 1)[0]
    FN_indx = np.where((edge_true == 1) & (edge_hat == 0) == 1)[0]

    theta_true = Theta[mask]
    theta_hat = Theta_hat[mask]

    norm_total = np.linalg.norm(theta_true - theta_hat)**2
    norm_TP = np.sum((theta_true[TP_indx]-theta_hat[TP_indx])**2)
    norm_FP = np.sum((theta_true[FP_indx]-theta_hat[FP_indx])**2)
    norm_FN= np.sum((theta_true[FN_indx]-theta_hat[FN_indx])**2)

    return norm_total, norm_TP, norm_FP, norm_FN

In [ ]:
one, two, three, four = compute_norm_test(Theta, Theta_hat)
print(one)
print(two + three + four)

2014.4127524530506
2014.4127524530504
